<a href="https://colab.research.google.com/github/Sparshsing/ai_timelapse/blob/main/time_lapse_Google_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Generate smooth Time lapse from multiple sequencial images

In [1]:
!git clone https://github.com/Sparshsing/ai_timelapse.git

Cloning into 'ai_timelapse'...
remote: Enumerating objects: 107, done.
remote: Total 107 (delta 0), reused 0 (delta 0), pack-reused 107 (from 1)
Receiving objects: 100% (107/107), 83.16 MiB | 9.48 MiB/s, done.
Resolving deltas: 100% (31/31), done.
Updating files: 100% (56/56), done.


### Instructions

1. Select runtime as T4 GPU from Settings.
2. Change the input and output paths and other settings. You can create a new folder inside ai_timelapse/sample_data and upload you image files. [After running the above cell]
3. Run the notebook. Notebook will automatically download the github repository, install requirements and download models.

In [2]:
# change directory to ai_timelapse
%cd ai_timelapse

/content/ai_timelapse


In [3]:
import os
os.getcwd()

'/content/ai_timelapse'

In [4]:
# check GPU (select T4 runtime in settings if error)
!nvidia-smi

Tue Apr 22 13:01:25 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   50C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [5]:
# assert that python version is less than 3.12
import sys
assert sys.version_info < (3, 12), "Python version must be less than 3.12"

In [6]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to

In [8]:
!pip uninstall -y numpy

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4


In [9]:
!pip install numpy==2.0.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 67.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mediapipe 0.10.21 requires numpy<2, but you have numpy 2.0.1 which is incompatible.
ydf 0.11.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 4.25.6 which is incompatible.


In [10]:
import tensorflow as tf

In [11]:
assert len(tf.config.list_physical_devices('GPU')) > 0, "No GPU available"

## Settings

In [33]:
INPUT_DIR = 'sample_data/shahrukh-khan'
OUTPUT_DIR = 'output_frames/shahrukh-khan'
REFERENCE_IMG_NAME = '1998_kuch_kuch_hota_hai.jpg'  # reference image for face tilt and lighting

preprocess_faces = True
FIX_LIGHTING = False  # to make images have a similar color
equal_intervals = True  # whether to use equal no of frames between images or decide based on date

# applicable when equal_intervals is False
# desired gap between frames in seconds (difference between dates of frames)
# eg. set 3600 to generate 1 frame for every 1 hr gap in input images
min_frame_duration = 3600 * 24

In [ ]:

MAX_DIM = 1080

VIDEO_FPS = 24

# set no of interpolated frames between images
# higher for better quality. try 15.
MAX_INTERPOLATED_FRAMES = 3  # 1 less than powers of 2
MODEL_PATH = "pretrained_models/film_net/Style/saved_model"
VALID_IMG_FORMATS = ('.png', '.jpg', '.jpeg', '.bmp')

In [25]:
if MAX_INTERPOLATED_FRAMES + 1 not in (2, 4, 8, 16, 32, 64, 128):
    raise ValueError("MAX_INTERPOLATED_FRAMES + 1 must be a power of 2")

In [26]:
import os
from pathlib import Path
import numpy as np
import tempfile
import tensorflow as tf
import mediapy
import cv2
from PIL import Image, ExifTags
import re

from natsort import natsorted
import shutil
from tqdm import tqdm
from datetime import datetime
import subprocess
import math
import mediapy as media
import sys
from typing import Generator, Iterable, List, Optional


from eval import interpolator, util
import fix_images

## Config

In [27]:
# Download the pretrained model
import gdown
os.makedirs('pretrained_models/film_net/Style/saved_model', exist_ok=True)

if os.path.exists('pretrained_models/film_net/Style/saved_model/saved_model.pb'):
    print('Model already downloaded')
else:
  folder_url = 'https://drive.google.com/drive/folders/1i9Go1YI2qiFWeT5QtywNFmYAA74bhXWj'
  gdown.download_folder(folder_url, output='pretrained_models/film_net/Style/saved_model', quiet=False)

Model already downloaded


## Plan

In [28]:
# 1. Preprocess images
# 2. for succesive pairs of images:
# 3.     generate interpolated frames
# 4.     add interpolated frames to the output directory
# 5. generate video from output directory

## Utility Functions

In [29]:

# create empty output directory
def create_empty_dir(dir_name):
  if os.path.exists(dir_name):
    shutil.rmtree(dir_name)
  os.makedirs(dir_name)


def resize_and_save(input_path, output_path, size):
  if input_path.lower().endswith(VALID_IMG_FORMATS):
      img = Image.open(input_path)
      img = img.resize(size, Image.Resampling.LANCZOS)
      img.save(output_path)
  else:
      raise ValueError(f'Unsupported file format: {input_path}')


def get_sorted_images(input_dir: str) -> Path:
    input_dir = Path(input_dir)
    image_files = [
        f.as_posix() for f in input_dir.iterdir()
        if f.is_file() and f.suffix.lower() in VALID_IMG_FORMATS
    ]

    # Sort files naturally
    image_files = natsorted(image_files)

    # image_files = natsorted([f for f in os.listdir(input_dir) if f.lower().endswith(VALID_IMG_FORMATS)])
    if not image_files:
        raise Exception("No images found in the directory.", input_dir)
    return image_files


# prepare input data
def prepare_images(input_dir, processed_input_dir, preprocess_faces, size, reference_img_name=None, fix_lighting=False):
    files = get_sorted_images(input_dir)

    # find the reference image
    if reference_img_name == '':
        reference_path = files[0]
    else:
        reference_path = None
        for f in files:
            image_path = Path(f)
            if image_path.name == reference_img_name:
                reference_path = image_path.with_name(reference_img_name)
                break
            elif str(image_path) == reference_img_name:
                reference_path = image_path
                break
            else:
                continue
        if reference_path is None:
            raise Exception('Reference image not found')

    print('reference_image', str(reference_path))

    for f in tqdm(files, desc='Preparing images'):
        output_file = (Path(processed_input_dir) / Path(f).name).as_posix()
        if preprocess_faces:
            fix_images.process_face_image(
                f,
                output_file,
                reference_path=str(reference_path),
                background_color=(255, 255, 255),  # White background
                save_bbox_preview=False,
                fix_lighting=fix_lighting
                )
        else:
            resize_and_save(f, output_file, size)


def get_new_size(image_path: str, max_dim: int = MAX_DIM) -> np.ndarray:
    """Resize the image so that the maximum dimension is `max_dim`."""
    image = cv2.imread(image_path)
    h, w = image.shape[:2]
    if h > w:
        new_h = max_dim
        new_w = int(w * new_h / h)
    else:
        new_w = max_dim
        new_h = int(h * new_w / w)
    return (new_w, new_h)


def string_to_timestamp(datetime_str):
    # Try yyyymmdd_hhmmss pattern
    pattern1 = r'(\d{8}_\d{6})'
    match = re.search(pattern1, datetime_str)
    if match:
        try:
            return datetime.strptime(match.group(1), "%Y%m%d_%H%M%S")
        except ValueError:
            pass

    # Try yyyymmdd pattern
    pattern2 = r'(\d{8})'
    match = re.search(pattern2, datetime_str)
    if match:
        try:
            return datetime.strptime(match.group(1), "%Y%m%d")
        except ValueError:
            pass

    raise ValueError("No valid date found in string. Expected format: YYYYMMDD or YYYYMMDD_HHMMSS")


def timestamp_to_string(timestamp):
    # Convert the timestamp number to a datetime object
    dt = datetime.fromtimestamp(timestamp)
    # Format the datetime object to the desired string format
    formatted_string = dt.strftime("%Y%m%d_%H%M%S")
    return formatted_string


def choose_evenly_spaced_timestamps(timestamps, k):
    # Ensure we have at least k timestamps
    if k > len(timestamps):
        raise ValueError("k cannot be greater than the number of timestamps")

    # Calculate the interval between timestamps
    n = len(timestamps)
    interval = (n - 1) / (k - 1)

    # Select the timestamps
    chosen_timestamps = []
    for i in range(k):
        index = round(i * interval)
        chosen_timestamps.append(timestamps[index])

    return chosen_timestamps


def save_video(frames, out_path):
    ffmpeg_path = util.get_ffmpeg_path()
    mediapy.set_ffmpeg(ffmpeg_path)
    mediapy.write_video(out_path, frames, fps=30)


def save_frames(frames, output_dir, format='jpg'):
    """
    Save interpolated frames to the specified output directory and return the output paths.
    Args:
        frames: List of image arrays
        output_dir: Directory to save frames
        format: Image format to save (jpg/png)
    Returns:
        list: List of file paths where the frames are saved.
    """
    output_paths = []
    for idx, frame in enumerate(frames):
        output_path = os.path.join(output_dir, f'frame_{idx:06d}.{format}')
        util.write_image(output_path, frame)
        output_paths.append(output_path)
    return output_paths



def write_video_from_images(image_dir, output_path, fps):
    image_files = get_sorted_images(image_dir)

    # Read the first image to get the dimensions
    first_image_path = image_files[0]
    first_frame = cv2.imread(first_image_path)
    height, width, layers = first_frame.shape

    # Initialize the video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Specify the codec
    video_writer = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    for image_file in image_files:
        frame = cv2.imread(image_file)
        if frame is None:
            print(f"Skipping {image_file}, cannot read image.")
            continue
        video_writer.write(frame)

    video_writer.release()
    print(f"Video saved to: {output_path}")

In [30]:

def interpolate_frames(image_1, image_2, times_to_interpolate, interpolator):
  input_frames = [str(image_1), str(image_2)]

  frames = list(
      util.interpolate_recursively_from_files(
          input_frames, times_to_interpolate, interpolator))
  return frames

## Main

In [31]:
# Preprocess images

processed_input_dir = tempfile.mkdtemp()
temp_interpolated_frames_dir = tempfile.mkdtemp()

files = get_sorted_images(INPUT_DIR)
for f in files:
    print(Path(f).name)


1992_deewana.jpg
1993_baazigar.jpg
1995_ddlj.jpg
1998_kuch_kuch_hota_hai.jpg
2003_kal_ho_na_ho.jpeg


In [34]:
new_size = get_new_size(files[0])
create_empty_dir(processed_input_dir)
prepare_images(INPUT_DIR, processed_input_dir, preprocess_faces, new_size, reference_img_name=REFERENCE_IMG_NAME, fix_lighting=FIX_LIGHTING)
files = get_sorted_images(processed_input_dir)


reference_image sample_data/shahrukh-khan/1998_kuch_kuch_hota_hai.jpg


Preparing images:   0%|          | 0/5 [00:00<?, ?it/s]Downloading data from 'https://github.com/danielgatis/rembg/releases/download/v0.0.0/u2net.onnx' to file '/root/.u2net/u2net.onnx'.

  0%|                                               | 0.00/176M [00:00<?, ?B/s]
  0%|                                       | 66.6k/176M [00:00<04:31, 649kB/s]
  0%|                                       | 394k/176M [00:00<01:21, 2.17MB/s]
  1%|▎                                     | 1.67M/176M [00:00<00:25, 6.96MB/s]
  4%|█▎                                    | 6.20M/176M [00:00<00:07, 22.0MB/s]
  6%|██                                    | 9.68M/176M [00:00<00:06, 26.6MB/s]
  8%|███                                   | 14.2M/176M [00:00<00:04, 32.8MB/s]
 10%|███▉                                  | 18.4M/176M [00:00<00:04, 36.1MB/s]
 13%|█████                                 | 23.6M/176M [00:00<00:03, 40.8MB/s]
 16%|██████                                | 28.3M/176M [00:00<00:03, 43.0MB/s]
 19%|███████

In [35]:
print('preprocessed images in', processed_input_dir)
print('file count', len(files))

preprocessed images in /tmp/tmpqjc2y8sz
file count 5


In [36]:
if not equal_intervals:
    for file in files:

      timestamp = string_to_timestamp(Path(file).stem)
      print(f"Filename: {file}, Datetime: {timestamp}")
else:
   for file in files:
      print(f"Filename: {file}")

Filename: /tmp/tmpqjc2y8sz/1992_deewana.jpg
Filename: /tmp/tmpqjc2y8sz/1993_baazigar.jpg
Filename: /tmp/tmpqjc2y8sz/1995_ddlj.jpg
Filename: /tmp/tmpqjc2y8sz/1998_kuch_kuch_hota_hai.jpg
Filename: /tmp/tmpqjc2y8sz/2003_kal_ho_na_ho.jpeg


In [37]:
# Iterate through pairs of consecutive frames

interpolator_model = interpolator.Interpolator(MODEL_PATH, None)
print('Total frames:', len(files))
create_empty_dir(OUTPUT_DIR)
for i in range(len(files) - 1):
    print('processing frame', i)
    start_filename = files[i]
    end_filename = files[i + 1]

    if not equal_intervals:
        start_time = string_to_timestamp(Path(start_filename).stem)
        end_time = string_to_timestamp(Path(end_filename).stem)

    # Calculate times_to_interpolate for the required number of intermediate frames
    if equal_intervals:
          num_frames_needed = MAX_INTERPOLATED_FRAMES
    else:
          # Calculate the time difference in hours
          time_diff = (end_time - start_time).total_seconds()
          num_frames_needed = round(time_diff / min_frame_duration) - 1
          num_frames_needed = min(num_frames_needed, MAX_INTERPOLATED_FRAMES)
    times_to_interpolate = math.ceil(math.log2(num_frames_needed+1))  # n_intermediate = 2^k - 1

    frame_1 = start_filename
    frame_2 = end_filename
    output_frames = interpolate_frames(frame_1, frame_2, times_to_interpolate, interpolator_model)

    create_empty_dir(temp_interpolated_frames_dir)
    save_frames(output_frames, temp_interpolated_frames_dir)
    interpolated_files = get_sorted_images(temp_interpolated_frames_dir)

    if not equal_intervals:
        timestamps = np.linspace(start_time.timestamp(), end_time.timestamp(), len(interpolated_files))
        # find required frames from generated frames (can be extra)
        chosen_timestamps = choose_evenly_spaced_timestamps(timestamps, num_frames_needed+2)

        for filename, timestamp in zip(interpolated_files, timestamps):
            if timestamp not in chosen_timestamps:
                continue
            new_filename = Path(OUTPUT_DIR) / f"{timestamp_to_string(timestamp)}{Path(filename).suffix}"
            shutil.copyfile(filename, new_filename)
    else:
        for interp_index, filename in enumerate(interpolated_files[:-1]):
          new_filename = Path(OUTPUT_DIR) / f"{Path(start_filename).stem}_{interp_index:04d}{Path(filename).suffix}"
          shutil.copyfile(filename, new_filename)
          # include end frame in last iteration
          if i == len(files) - 2:
              new_filename = Path(processed_input_dir) / f"{Path(end_filename).stem}_0000{Path(filename).suffix}"
              shutil.copyfile(end_filename, new_filename)



Total frames: 5
processing frame 0


100%|█████████████████████████████████████████████████████████████████| 3/3 [00:30<00:00, 10.27s/it]


processing frame 1


100%|█████████████████████████████████████████████████████████████████| 3/3 [00:04<00:00,  1.35s/it]


processing frame 2


100%|█████████████████████████████████████████████████████████████████| 3/3 [00:04<00:00,  1.36s/it]


processing frame 3


100%|█████████████████████████████████████████████████████████████████| 3/3 [00:04<00:00,  1.37s/it]


In [41]:

print("TensorFlow version:", tf.__version__)
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))

TensorFlow version: 2.18.0
Num GPUs Available: 1


In [ ]:

# generate video from output directory
output_path = str(OUTPUT_DIR) + '.mp4'

write_video_from_images(OUTPUT_DIR, output_path, VIDEO_FPS)

Video saved to: output_frames/shahrukh-khan.mp4


In [40]:
# todo: try GAN using https://github.com/eladrich/pixel2style2pixel